In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold	
from tensorflow.keras.utils import to_categorical

#將 train.csv 讀入pandas的DataFrame
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
train_x = train.drop(['label'], axis=1)     # 從train取出圖像資料 
train_y = train['label']                    # 從train取出正確答案
test_x = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
 # 將train的資料分為訓練資料跟驗證資料。
kf = KFold(n_splits=4, shuffle=True, random_state=123)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
 # 將圖像的像素值除以255.0，收斂在0 ~ 1.0範圍內，並變換為numpy.array
tr_x, va_x = np.array(tr_x / 255.0), np.array(va_x / 255.0)
# 將二維矩陣的圖像數據變換成( 高度 = 28, 寬度 = 28, 通道 = 1) 的三維矩陣。
# 灰階圖片的通道值為1
tr_x = tr_x.reshape(-1,28,28,1)
va_x = va_x.reshape(-1,28,28,1)

# 將正確答案轉換為One-hot encoding呈現
tr_y = to_categorical(tr_y, 10)
va_y = to_categorical(va_y, 10)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten # core layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D # convolution layers
 # 生成Sequential 物件
model = Sequential()
 # 第1層: 卷積層
model.add(Conv2D( filters=32,
                 kernel_size=(5,5),
                 padding='same',
                 activation='relu',
                 input_shape=(28,28,1)))
 # 第2層: 卷積層
model.add(Conv2D( filters = 64,
                 kernel_size = (7,7),
                 padding='same',
                 activation='relu'))
 # 第3層: 池化層
model.add(MaxPooling2D(pool_size=(2,2)))
 # 丟棄法
model.add(Dropout(0.5))

# 第4層: 卷積層
model.add(Conv2D( filters=64,
                 kernel_size=(5,5),
                 padding='same',
                 activation='relu'))

# 第5層: 卷積層
model.add(Conv2D( filters = 32,
                 kernel_size = (3,3),
                 padding='same',
                 activation='relu'))

# 第6層: 池化層
model.add(MaxPooling2D(pool_size=(2,2)))

# 丟棄法
model.add(Dropout(0.55))

# 扁平層
model.add(Flatten())

#第7層: 全連接層
model.add(Dense(700, activation='relu'))
model.add(Dropout(0.3))

#第8層: 全連接層
model.add(Dense(150, activation='relu'))
model.add(Dropout(0.35))

#第10層: 輸出層
model.add(Dense(10, activation = "softmax"))

# 編譯模型
# 優化器為Adam
momentum = 0.5
model.compile(loss="categorical_crossentropy",
              optimizer='adam',
              metrics=["accuracy"])


In [ ]:
# 批次大小
batch_size = 100
# 訓練次數
epochs = 20
# 執行學習
history = model.fit(tr_x, tr_y,                    # 訓練資料
                    batch_size=batch_size,         # 批次大小
                    epochs=epochs,                 # 訓練次數
                    verbose=1,                     # 輸出訓練的狀況
                    validation_data=(va_x,va_y)    # 驗證資料
                   )


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# 設定繪圖尺寸
plt.figure(figsize=(15, 6))
# 將圖形縮小、讓圖形之間保有空間
plt.subplots_adjust(wspace=0.2)
# 在1 × 2格線左方 (1,2,1)範圍內繪圖
plt.subplot(1, 2, 1)
# 描繪出訓練資料的損失（錯誤率）
plt.plot(history.history['loss'],
         label='training',
         linestyle = '--') 
# 描繪出驗證資料的損失（錯誤率） 
plt.plot(history.history['val_loss'],
         label='validation') 
plt.ylim(0, 1)          # y 軸範圍 
plt.legend()            # 顯示圖例 
plt.grid()              # 顯示格線 
plt.xlabel('epoch')     # x 軸標籤
plt.ylabel('loss')      # y 軸標籤
#在1 × 2格線右方(1,2,2)範圍內繪圖
plt.subplot(1, 2, 2)
# 描繪出訓練資料的準確率
plt.plot( history.history['accuracy'],
         linestyle = '--',
         label='training')
# 描繪出驗證資料的準確率
plt.plot(history.history['val_accuracy'],
         label='validation')
plt.ylim(0.5, 1)         # y 軸範圍
plt.legend()             # 顯示圖例
plt.grid()               # 顯示格線
plt.xlabel('epoch')      # x 軸標籤
plt.ylabel('acc')        # y 軸標籤
plt.show()
